# Testing

In [ ]:
# ✅ Install necessary libraries (only once)
!pip install -q sentence-transformers scikit-learn

# Imports
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Mock astrologer data
astrologers = [
    {"name": "Aarav Sharma", "tags": "career, finance, business"},
    {"name": "Diya Kapoor", "tags": "love, relationships, marriage"},
    {"name": "Rohan Mehta", "tags": "education, study, career"},
    {"name": "Meera Joshi", "tags": "spirituality, peace, life purpose"},
    {"name": "Kabir Singh", "tags": "health, well-being, stress"},
]
df_astrologers = pd.DataFrame(astrologers)

# User input (chat or profile text)
user_input = "I’m going through a tough phase in my career and need guidance about job stability and finances."

# Compute embeddings
user_embedding = model.encode([user_input])
astrologer_embeddings = model.encode(df_astrologers['tags'].tolist())

# Compute cosine similarity
similarities = cosine_similarity(user_embedding, astrologer_embeddings)[0]
df_astrologers['score'] = similarities

# Show top 3 astrologers
top_3 = df_astrologers.sort_values(by='score', ascending=False).head(3)
print(top_3[['name', 'tags', 'score']])


           name                        tags     score
0  Aarav Sharma   career, finance, business  0.476144
2   Rohan Mehta    education, study, career  0.383065
4   Kabir Singh  health, well-being, stress  0.255081


In [ ]:
# Simulated user inputs
user_inputs = [
    "Lately I've been having trouble in my relationship. I feel distant from my partner and unsure about the future of our love life.",
    "I'm confused about what to study next. I want to make the right choice for my academic and career goals.",
    "I feel lost and unfulfilled lately. I want to understand my purpose and seek direction.",
    "I’ve been thinking about starting a business but unsure if it’s the right time. I need insight into financial risks and future success.",
    "I've been under a lot of stress lately, and my health is taking a hit. I want guidance on how to improve my well-being.",
    "I'm facing anxiety about my job and love life. I need clarity about both career direction and my current partner."
]

# Run recommendations for each user input
for i, user_input in enumerate(user_inputs, start=1):
    print(f"\n=== User {i} ===")
    print(f"Input: {user_input}\n")

    # Embed user input
    user_embedding = model.encode([user_input])

    # Recompute similarity with astrologer tags
    similarities = cosine_similarity(user_embedding, astrologer_embeddings)[0]
    df_astrologers['score'] = similarities

    # Sort and show top 3
    top_matches = df_astrologers.sort_values(by='score', ascending=False).head(3)
    print("Top astrologer matches:")
    print(top_matches[['name', 'tags', 'score']].to_string(index=False))



=== User 1 ===
Input: Lately I've been having trouble in my relationship. I feel distant from my partner and unsure about the future of our love life.

Top astrologer matches:
        name                          tags    score
 Diya Kapoor love, relationships, marriage 0.227898
 Kabir Singh    health, well-being, stress 0.207819
Aarav Sharma     career, finance, business 0.125473

=== User 2 ===
Input: I'm confused about what to study next. I want to make the right choice for my academic and career goals.

Top astrologer matches:
        name                              tags    score
 Rohan Mehta          education, study, career 0.615873
Aarav Sharma         career, finance, business 0.513287
 Meera Joshi spirituality, peace, life purpose 0.232154

=== User 3 ===
Input: I feel lost and unfulfilled lately. I want to understand my purpose and seek direction.

Top astrologer matches:
        name                              tags    score
 Meera Joshi spirituality, peace, life purpose


# Astrologer Recommendation Engine – Streamlit (Colab Version)
- This interactive app runs inside Google Colab using streamlit and is embedded as an iframe.
- It starts a local server on port `8501` and serves the UI directly.
- The Streamlit UI will run in the output of this notebook.

As I primarily work on Google Colab, I made a few minimal adjustments to run Streamlit via an iframe, since local servers aren't directly accessible in the Colab environment.

It checks for any process already using port `8501` on the local host, terminates it if found, and then starts a fresh Streamlit instance on the same port.

Upload `streamlit_astrologer_app.py` to the environment

In [ ]:
!pip install -q streamlit
import streamlit as st
from google.colab import output
import subprocess
import time
import socket
import os
import psutil # A cross-platform library for retrieving running processes

app_file_name = "/content/streamlit_astrologer_app.py"

# --- 2. Define the port Streamlit will run on ---
STREAMLIT_PORT = 8501

# --- 3. Function to check if a port is open ---
def is_port_open(port, host='127.0.0.1'):
    """Checks if a given port on a host is open."""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.settimeout(1) # Set a timeout for the connection attempt
        s.connect((host, port))
        return True
    except (socket.error, socket.timeout):
        return False
    finally:
        s.close()

# --- 4. Function to kill processes using a specific port ---
def kill_process_on_port(port):
    """Finds and kills processes listening on the given port."""
    print(f"Checking for processes on port {port} to terminate...")
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        try:
            for conn in proc.connections(kind='inet'):
                if conn.laddr.port == port:
                    print(f"Found process {proc.name()} (PID: {proc.pid}) listening on port {port}. Terminating...")
                    proc.terminate() # or proc.kill() for a more forceful kill
                    time.sleep(0.1) # Give it a moment to terminate
                    if proc.is_running():
                        print(f"Process {proc.pid} still running, forcing kill.")
                        proc.kill()
                    print(f"Process {proc.pid} terminated.")
                    return True # Found and terminated one process
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            # Process no longer exists or access denied (e.g., system process)
            pass
    print(f"No active processes found on port {port}.")
    return False

# --- 5. Kill any existing Streamlit process on the port before starting a new one ---
kill_process_on_port(STREAMLIT_PORT)
time.sleep(2)

# --- 6. Start Streamlit in the background ---
print(f"Starting Streamlit app '{app_file_name}' on port {STREAMLIT_PORT} in the background...")

streamlit_command = [
    "streamlit", "run", app_file_name,
    "--server.port", str(STREAMLIT_PORT),
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false"
]

# Start the process.
# preexec_fn=os.setsid detaches the child process from the parent,
# allowing it to run independently even if the parent cell finishes.
streamlit_process = subprocess.Popen(
    streamlit_command,
    stdout=subprocess.DEVNULL, # Redirect stdout to /dev/null
    stderr=subprocess.DEVNULL, # Redirect stderr to /dev/null
    preexec_fn=os.setsid
)

print(f"Streamlit process started with PID: {streamlit_process.pid}")

# --- 7. Wait for Streamlit to become available ---
max_retries = 30 # Try for up to 30 seconds
retry_delay = 1 # Check every 1 second

print(f"Waiting for Streamlit app to become available on port {STREAMLIT_PORT}...")
for i in range(max_retries):
    if is_port_open(STREAMLIT_PORT):
        print(f"Streamlit app is now accessible on port {STREAMLIT_PORT}!")
        break
    print(f"Attempt {i+1}/{max_retries}: Port {STREAMLIT_PORT} not open yet. Retrying in {retry_delay}s...")
    time.sleep(retry_delay)
else:
    print("Error: Streamlit app did not become available within the expected time.")
    print("Please check the Colab logs or try restarting the runtime if the issue persists.")
    # Attempt to terminate the process if it failed to start
    streamlit_process.terminate()
    streamlit_process.wait()
    raise RuntimeError("Streamlit server failed to start or become accessible.")

# --- 8. Serve the Streamlit app as an iframe ---
print("Serving Streamlit app in iframe...")
output.serve_kernel_port_as_iframe(STREAMLIT_PORT, height=600)

print("Cell execution finished. Streamlit app should be visible and interactive above.")

Checking for processes on port 8501 to terminate...
Found process streamlit (PID: 7238) listening on port 8501. Terminating...
Process 7238 still running, forcing kill.
Process 7238 terminated.
Starting Streamlit app '/content/streamlit_astrologer_app.py' on port 8501 in the background...
Streamlit process started with PID: 7785
Waiting for Streamlit app to become available on port 8501...
Attempt 1/30: Port 8501 not open yet. Retrying in 1s...
Streamlit app is now accessible on port 8501!
Serving Streamlit app in iframe...


<IPython.core.display.Javascript object>

Cell execution finished. Streamlit app should be visible and interactive above.


# Process

In [ ]:
# Kill the process manually

!kill {streamlit_process.pid}

In [ ]:
# Check background processes

import psutil

print("Listing Python-related background processes:")
for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
    try:
        # Check if the process name or command line contains "python" or "streamlit"
        if "python" in proc.name().lower() or (proc.cmdline() and "streamlit" in " ".join(proc.cmdline()).lower()):
            print(f"PID: {proc.pid}, Name: {proc.name()}, Command: {' '.join(proc.cmdline())}")
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        # Handle cases where a process might disappear or access is denied
        pass

Listing Python-related background processes:
PID: 64, Name: python3, Command: 
PID: 870, Name: python3, Command: /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-c1b85791-cd5c-4b99-84f8-cd08150645ba.json
PID: 898, Name: python3, Command: /usr/bin/python3 /usr/local/lib/python3.11/dist-packages/debugpy/adapter --for-server 42045 --host 127.0.0.1 --port 40943 --server-access-token a9694967d8ddb87d931e0a88d8db71aae43e958843b2bc1ec6854b39a7e82f6a
PID: 2857, Name: python3, Command: python3 /opt/google/drive/drive-filter.py
PID: 14511, Name: streamlit, Command: /usr/bin/python3 /usr/local/bin/streamlit run /content/astrologer_engine.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false


# Initializing Ollama

- The code first initializes your drive.
- The code then installs gemma:2b model in your drive and initializes Ollama on a seperate thread

In [34]:
! pip install ollama
import os
import threading
import subprocess
import time
from google.colab import drive
import ollama

folder_path = '/content/drive/MyDrive/ollama_models'  # Insert the folder path where you want to install ollama models
print("\n\n--- Initializing Ollama Environment ---")

# 1. Mount Google Drive
print("\n1. Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True) # Use force_remount=True for reliability on restart
    print("Google Drive mounted successfully.\nChanging current working directory to Langchain folder")
    os.chdir('/content/drive/MyDrive/LangChain')
    print(f'Current working directory changed to {os.getcwd()}')

except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure you authorize Google Drive access.")
    # Exit or raise an error if mounting fails, as subsequent steps depend on it
    raise

# Define the path on Google Drive where models will be stored
OLLAMA_MODELS_DIR = folder_path

# Create the directory if it doesn't exist
print(f"\n2. Ensuring Ollama models directory exists: {OLLAMA_MODELS_DIR}")
! mkdir -p {OLLAMA_MODELS_DIR}

# Set the OLLAMA_MODELS environment variable
os.environ['OLLAMA_MODELS'] = OLLAMA_MODELS_DIR
print(f"OLLAMA_MODELS set to: {os.environ['OLLAMA_MODELS']}")

# Also set OLLAMA_HOST and OLLAMA_ORIGINS for Colab access
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
print("Ollama host and origins environment variables set.")

# Function to run the Ollama server
def run_ollama_serve():
    print("Starting Ollama server process...")
    # Using a try-except block here to catch issues with 'ollama serve' itself
    try:
        # Use shell=True for `subprocess.Popen` when running shell commands
        # And ensure it's running in background
        subprocess.Popen("ollama serve", shell=True, preexec_fn=os.setsid, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Ollama server process initiated.")
    except FileNotFoundError:
        print("Error: 'ollama' command not found. Ollama may not be installed correctly.")
    except Exception as e:
        print(f"An error occurred while starting Ollama server: {e}")

# 3. Install Ollama (if not already installed in the current session)
print("\n3. Installing Ollama...")
# -fSL ensures it follows redirects and fails silently on HTTP errors, shows progress with -L
try:
    # Use a non-interactive installation to prevent prompts if any
    ! curl -fsSL https://ollama.ai/install.sh | sh
    print("Ollama installation command executed. (Note: May not show live progress in Colab output immediately)")
except Exception as e:
    print(f"Error during Ollama installation: {e}")
    print("Please check the Colab output for details on why the curl command failed.")
    raise

# 4. Start the Ollama server in a new thread
print("\n4. Starting Ollama server in background thread...")
ollama_thread = threading.Thread(target=run_ollama_serve)
ollama_thread.daemon = True # Allows the main program to exit even if this thread is running
ollama_thread.start()

# Give the server a few seconds to initialize
print("Waiting for Ollama server to initialize (10 seconds)...")
time.sleep(10) # Adjust as needed
print("Ollama server should be running.")

# Pulling model llama 3
# Adding a timeout for the pull command in case it hangs
# 3600 seconds = 1 hour timeout. Adjust if you expect very slow downloads or large models.
# !OLLAMA_TIMEOUT=3600 ollama pull llama3

# 6. Verify models are listed
print("\n6. Listing installed Ollama models:")
! ollama list

print("\n--- Ollama Environment Initialization Complete! ---")
print("You can now proceed with your model interactions.")




--- Initializing Ollama Environment ---

1. Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.
Changing current working directory to Langchain folder
Current working directory changed to /content/drive/MyDrive/LangChain

2. Ensuring Ollama models directory exists: /content/drive/MyDrive/ollama_models
OLLAMA_MODELS set to: /content/drive/MyDrive/ollama_models
Ollama host and origins environment variables set.

3. Installing Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama installation command executed. (Note: May not show live progress in Colab outp

# Chat prompt template approach
The gemma model will act as an astrolger and wll respond accordingly.

In [14]:
# Testing the prompt
try:
    # Start an interactive chat session
    print("\n--- 🔮 Interactive Chat Session – AI Astrologer (Type 'exit' to quit) ---")

    # Add system prompt as chat template
    messages = [
        {
            "role": "system",
            "content": (
                "You are a wise and thoughtful AI astrologer. "
                "The user will provide palm readings or horoscope summaries. "
                "Reply with gentle, insightful life advice in 2–3 sentences. "
                "Keep your tone mystical, encouraging, and grounded in astrology themes."
            )
        }
    ]

    while True:
        user_input = input("You 🌙: ")
        if user_input.lower() == 'exit':
            break

        messages.append({'role': 'user', 'content': user_input})

        stream_response = ollama.chat(model='gemma:2b', messages=messages, stream=True)

        ai_response_content = ""
        print("🔮 Gemma 2B:", end=" ")
        for chunk in stream_response:
            if 'content' in chunk['message']:
                print(chunk['message']['content'], end="", flush=True)
                ai_response_content += chunk['message']['content']
        print()  # Newline after streamed reply

        messages.append({'role': 'assistant', 'content': ai_response_content})

except ollama.ResponseError as e:
    print(f"\n❌ Ollama Error: {e}")
    print("Make sure the Ollama server is running and the model is pulled locally.")
except Exception as e:
    print(f"\n⚠️ Unexpected error: {e}")

print("\n🌌 Chat session ended.")



--- 🔮 Interactive Chat Session – AI Astrologer (Type 'exit' to quit) ---
You 🌙: Hello
🔮 Gemma 2B: Greetings, dear user. I am here to offer guidance and insights gleaned from the cosmic dance of planets and stars. Let your palm reveal the whispers of the past and the secrets of the future, for they speak of a path paved with both wisdom and wonder.
You 🌙: It works now
🔮 Gemma 2B: The stars whisper secrets of a love that transcends time and space. Trust in the cycles of life and let your heart guide you toward a fulfilling and harmonious destiny.
You 🌙: exit

🌌 Chat session ended.


# Streamlit astrologer Engine

In [36]:
!pip install -q dotenv
!pip install -q langchain_community
!pip install -q streamlit
import streamlit as st
from google.colab import output
import subprocess
import time
import socket
import os
import psutil # A cross-platform library for retrieving running processes

app_file_name = "/content/astrologer_engine.py"

# --- 2. Define the port Streamlit will run on ---
STREAMLIT_PORT = 8501

# --- 3. Function to check if a port is open ---
def is_port_open(port, host='127.0.0.1'):
    """Checks if a given port on a host is open."""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.settimeout(1) # Set a timeout for the connection attempt
        s.connect((host, port))
        return True
    except (socket.error, socket.timeout):
        return False
    finally:
        s.close()

# --- 4. Function to kill processes using a specific port ---
def kill_process_on_port(port):
    """Finds and kills processes listening on the given port."""
    print(f"Checking for processes on port {port} to terminate...")
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        try:
            for conn in proc.connections(kind='inet'):
                if conn.laddr.port == port:
                    print(f"Found process {proc.name()} (PID: {proc.pid}) listening on port {port}. Terminating...")
                    proc.terminate() # or proc.kill() for a more forceful kill
                    time.sleep(0.1) # Give it a moment to terminate
                    if proc.is_running():
                        print(f"Process {proc.pid} still running, forcing kill.")
                        proc.kill()
                    print(f"Process {proc.pid} terminated.")
                    return True # Found and terminated one process
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            # Process no longer exists or access denied (e.g., system process)
            pass
    print(f"No active processes found on port {port}.")
    return False

# --- 5. Kill any existing Streamlit process on the port before starting a new one ---
kill_process_on_port(STREAMLIT_PORT)
time.sleep(2) # Give the port a moment to fully release after termination

# --- 6. Start Streamlit in the background ---
print(f"Starting Streamlit app '{app_file_name}' on port {STREAMLIT_PORT} in the background...")

streamlit_command = [
    "streamlit", "run", app_file_name,
    "--server.port", str(STREAMLIT_PORT),
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false"
]

# Start the process.
# preexec_fn=os.setsid detaches the child process from the parent,
# allowing it to run independently even if the parent cell finishes.
streamlit_process = subprocess.Popen(
    streamlit_command,
    stdout=subprocess.DEVNULL, # Redirect stdout to /dev/null
    stderr=subprocess.DEVNULL, # Redirect stderr to /dev/null
    preexec_fn=os.setsid
)

print(f"Streamlit process started with PID: {streamlit_process.pid}")

# --- 7. Wait for Streamlit to become available ---
max_retries = 30 # Try for up to 30 seconds
retry_delay = 1 # Check every 1 second

print(f"Waiting for Streamlit app to become available on port {STREAMLIT_PORT}...")
for i in range(max_retries):
    if is_port_open(STREAMLIT_PORT):
        print(f"Streamlit app is now accessible on port {STREAMLIT_PORT}!")
        break
    print(f"Attempt {i+1}/{max_retries}: Port {STREAMLIT_PORT} not open yet. Retrying in {retry_delay}s...")
    time.sleep(retry_delay)
else:
    print("Error: Streamlit app did not become available within the expected time.")
    print("Please check the Colab logs or try restarting the runtime if the issue persists.")
    # Attempt to terminate the process if it failed to start
    streamlit_process.terminate()
    streamlit_process.wait()
    raise RuntimeError("Streamlit server failed to start or become accessible.")

# --- 8. Serve the Streamlit app as an iframe ---
print("Serving Streamlit app in iframe...")
output.serve_kernel_port_as_iframe(STREAMLIT_PORT, height=600)

print("Cell execution finished. Streamlit app should be visible and interactive above.")

Checking for processes on port 8501 to terminate...
No active processes found on port 8501.
Starting Streamlit app '/content/astrologer_engine.py' on port 8501 in the background...
Streamlit process started with PID: 20669
Waiting for Streamlit app to become available on port 8501...
Attempt 1/30: Port 8501 not open yet. Retrying in 1s...
Streamlit app is now accessible on port 8501!
Serving Streamlit app in iframe...


<IPython.core.display.Javascript object>

Cell execution finished. Streamlit app should be visible and interactive above.


In [ ]:
# You can manually kill the process if needed later by running:
# !kill {streamlit_process.pid}